# Import libraries

In [ ]:
# Classic stuff
import numpy as np 
import matplotlib.pyplot as plk
import pandas as pd

# Import sklearn packages
from sklearn.svm import *
from sklearn.model_selection import *
from sklearn.pipeline import *
from sklearn.impute import *
from sklearn.preprocessing import *
import sklearn.metrics as metrics

# Load the dataset

In [ ]:
# Load the dataset
train_features = pd.read_csv('train_features.csv')
train_labels = pd.read_csv('train_labels.csv')
test_features = pd.read_csv('test_features.csv')

In [ ]:
# Encode the name of the labels
labels = ['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST',
         'LABEL_Alkalinephos', 'LABEL_Bilirubin_total', 
         'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2',
         'LABEL_Bilirubin_direct', 'LABEL_EtCO2']

labels_2 = ['LABEL_Sepsis']

labels_3 = ['LABEL_RRate','LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']

# Preparation of the dataset

In [ ]:
# This function can handle the time series and will return some features of it
def process_patient(patient_data):
    avgarray = np.nanmean(patient_data,axis=0)
    minarray = np.nanmin(patient_data,axis=0)
    maxarray = np.nanmax(patient_data,axis=0)
    vararray = np.nanvar(patient_data,axis=0)
    medarray = np.nanmedian(patient_data,axis=0)
    numarray = (~np.isnan(patient_data)).sum(axis=0)
    output = np.ravel(np.array([avgarray,minarray,maxarray,vararray,medarray,numarray]))
    return output

# This function will transform the time series into a normal dataset 
def compute_features_for_patient(X_train):
    output = []
    nbPatient = int(X_train.shape[0] / 12)
    for i in range(nbPatient):
        patient_data = X_train[12*i:12*(i+1),:]
        output.append(process_patient(patient_data))
    return np.array(output)

In [ ]:
# Preprocessing of the dataset
X_train = compute_features_for_patient(np.asarray(train_features)[:,3:])
X_train = np.insert(X_train,0, np.asarray(train_features.groupby(['pid'],as_index=False,sort=False).mean())[:,2],axis=1)
X_test  = compute_features_for_patient(np.asarray(test_features)[:,3:])
X_test  = np.insert(X_test,0, np.asarray(test_features.groupby(['pid'],as_index=False,sort=False).mean())[:,2],axis=1)

# Subtasks 1-3

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingRegressor

output = []

#Subtask 1
for i in labels:
    pipeline = make_pipeline(SimpleImputer(strategy = "mean"),StandardScaler(),HistGradientBoostingClassifier())
    pipeline.fit(X_train,np.asarray(train_labels[i]))
    output.append(pipeline.predict_proba(X_test)[:,1]) 

#Subtask 2
pipeline_2 = make_pipeline(SimpleImputer(strategy = "mean"),StandardScaler(),HistGradientBoostingClassifier())
pipeline_2.fit(X_train, np.asarray(train_labels[labels_2]))
output.append(pipeline_2.predict_proba(X_test)[:,1])  

#Subtask 3
for i in labels_3:
    pipeline_3 = make_pipeline(SimpleImputer(strategy = "mean"),StandardScaler(),HistGradientBoostingRegressor())
    pipeline_3.fit(X_train,np.asarray(train_labels[i]))
    output.append(pipeline_3.predict(X_test))

output = np.array(output)

# Output the results

In [ ]:
# Create the dataframe
data = pd.DataFrame(np.transpose(output), columns=labels+labels_2+labels_3)
pid = (compute_features_for_patient(np.asarray(test_features)))[:,0]

# Add the PID of the patient in the output
data.insert(0, 'pid', pid)

In [ ]:
# Output csv
data.to_csv('output.csv', index=False, header=True, float_format='%.3f')